# ONEWAY EXTRACTION

In [ ]:
import numpy as np

import pandas as pd # To structure and manipulated data in a DataFrame format
import geopandas as gpd # To work with spatial data in a DataFrame
from geopandas import GeoDataFrame # To create a GeoDataFrame from a DataFrame

import shapely
from shapely.geometry import shape, Point, LineString # To create line geometries that can be used in a GeoDataFrame

from geographiclib.geodesic import Geodesic

#Thư viện quy đổi tọa độ -> met
from haversine import haversine, Unit
import matplotlib.pyplot as plt
plt.rcParams['axes.xmargin'] = 0.1
plt.rcParams['axes.ymargin'] = 0.1
%matplotlib inline


# INPUT DATA

In [ ]:
day26 = 'data_pre_day26.csv'
day27 = 'data_pre.csv'
day_old = 'improved_preprocessing.csv'
day = day27
df= pd.read_csv(f'input/{day}')
df_lines = pd.read_csv('input/multipoint_new.csv')

### Add column: trip 

In [ ]:
def add_trip_number(df):
    arr = df.loc[:, 'level_1'].values
    indices = np.where(arr == 0)[0]
    indices = np.append(indices, [len(df)])
    max_size = np.amax(np.diff(indices))
    duplicate_idx = np.repeat(indices, 2)[1:-1].reshape(-1, 2)

    def add_number(x):
        idx = np.where(indices == x[0])[0]
        result = np.full((x[1] - x[0],), idx)
        result = np.pad(result, (0, max_size - len(result)), 'constant', constant_values=-1)
        return result

    trip_number_result = np.apply_along_axis(add_number, 1, duplicate_idx)
    trip_number_result = trip_number_result[trip_number_result != -1]
    df['trip'] = trip_number_result
    return df

In [ ]:
def add_heading(df):
    def calculate_heading(df):
        heading_list = []
        for i in df.index.tolist()[:-1]:
            heading = Geodesic.WGS84.Inverse(df.y[i], df.x[i], df.y[i + 1], df.x[i + 1])['azi1'] % 360
            heading_list.append(heading)
        heading_list.append(heading)
        return heading_list
    
    heading_list = []
    for i in df.trip.unique():
        heading_list.extend(calculate_heading(df.loc[df.trip == i]))
    
    df['heading'] = heading_list
    return df

In [ ]:
# rename x, y cols
df.rename({'x':'y', 'y':'x'}, axis=1, inplace=True)
# rename label col
df_lines.rename({'label': 'trip'}, axis=1, inplace=True)

add_trip_number(df)
add_heading(df_lines)

In [ ]:
df.describe()

### Format DF

In [ ]:
# geometry = [Point(xy) for xy in zip(df.x, df.y)]
# # gdf = df.drop(['y', 'x'], axis=1)
# # gdf = gdf.sort_values('datetime', ascending = True)
# gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

# drop columns
df.drop(['level_1','speed','vehicletype', 'time_interval','distance'], axis=1, inplace=True)

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(['vehicle', 'datetime'], ascending=[True, True])
df.reset_index(drop=True, inplace=True)
df['heading_interval'] = 0

## Test data

In [ ]:
df_test = df.copy()

In [ ]:
df_test.head()

# EXTRACTING ONEWAY ATTRIBUTE BY HEADING

<img src="img/oneway_extraction_alg.png" />

Source: <a href="https://drive.google.com/file/d/1gJY6XpEbzMk6-yuhgRB8D9Vd7anJq4Pf/view">
    Automatically Deriving and Updating Attribute Road Data from Movement Trajectories (chapter 5)
    <a/>

## helper functions: get trajec's center point & find nearest points  

Get trajecs center point

In [ ]:
''' get_center_point() params:
input:
    gdf_single_trajec: geodataframe - a single trajectory
output:
    center_point: geodataframe - a single point
'''
def get_center_point(gdf_single_trajec):
    trajec_len = len(gdf_single_trajec)
    center = int(trajec_len/2)
    # get center point gdf
    center_point = gdf_single_trajec.iloc[center]
    
    return center_point

Find all points around center point

In [ ]:
''' find_nearest_point() params:
input:
    center_point: geodataframe - a single point
    df: dataframe - graph data
    eps: meter - radius (50m as default)
output:
    gdf_cluster: geodataframe - a group of point nearest center point
'''
def find_nearest_point(center_point, df, kd, eps=10):
    # extract cluster labeled
    nearest_pts_id = kd.query_ball_point((center_point.x, center_point.y),eps*10**-3)
    df_cluster = df.iloc[nearest_pts_id].copy()
    
#     # filter center point in df
#     center_point = (center_point.x, center_point.y)
#     df_cluster['haversine_dis'] = df_cluster.apply(lambda f: int(haversine((f.x, f.y), center_point, unit=Unit.METERS)), axis=1)
#     df_cluster = df_cluster[df_cluster.haversine_dis <= eps]
    return df_cluster



 


Add column: Heading interval

In [ ]:
''' add_heading_interval() params:
input:
    df_cluster: dataframe - a group of point nearest center point
    center_point: geodataframe - a single point
output:
    df_cluster: dataframe - add heading_interval column
'''
def add_heading_interval(df_cluster, center_point):
    df_cluster['heading_interval'] = df_cluster.heading.apply(lambda heading: abs(center_point.heading-heading) % 360 ) 
    return df_cluster

## Attribute Extraction of "oneway" implement

In [ ]:
''' detect_oneway() params:
input:
    gdf: geodataframe - all data
    trip_id: geodataframe - a sub gdf represent for a trip
    trips: list trip id
output:
    tag = 1, tag = -1 -> oneway
    tag = 0 -> no
    tag = -2 -> can't detect
'''
def detect_oneway(gdf, gdf_lines, kd, trip_id):
    tag = -2 # tag for
    # select trip from trip_id
    gdf_trip = gdf_lines[gdf_lines.trip == trip_id]    
    # SELECT RELATIVE DIRECTION FOR CLUSTER
    center_point = get_center_point(gdf_trip) # get mid point of a trip
    df_cluster = find_nearest_point(center_point, gdf, kd, eps=8)
    if df_cluster.empty:
        return tag, df_cluster     # return if cluster is empty
    df_cluster.drop_duplicates(subset=['trip'], inplace=True) # drop trip duplicates
    
    cluster_size = len(df_cluster)
    # ignore cluster if the size is less than 10 points
    size_thresh = 5
    if cluster_size >= size_thresh:
        print(f'> {size_thresh}')
        # add heading_interval
        add_heading_interval(df_cluster, center_point)
        print(f'heading {center_point.heading}')
        # 
        radian_thresh = 40
        similar = len(df_cluster.loc[(df_cluster.heading_interval <= radian_thresh) | (df_cluster.heading_interval >= (360-radian_thresh))])
        opposite = len(df_cluster.loc[(df_cluster.heading_interval >= (180-radian_thresh)) & (df_cluster.heading_interval <= (180+radian_thresh))])
        total = len(df_cluster)
        print(f'similar: {similar}')
        print(f'opposite: {opposite}')
        print(f'total: {total}')

        # tag = 1, tag = -1 -> oneway
        # tag = 0 -> no
        rate_thresh = 0.85
        if similar == 0 and opposite == 0:
            pass
        elif similar/total > rate_thresh:
            tag = 1
        elif opposite/total > rate_thresh:
            tag = -1
        else:
            tag = 0
    else:
        print(f'< {size_thresh}')
    return tag, df_cluster

## ONEWAY EXTRACTION

In [ ]:
# ## TEST 1 TRIP
# trip_id =106
# df_trip = df_lines[df_lines.trip == trip_id]    
# center_point = get_center_point(df_trip) # get mid point of a trip

# radius = 10*10**-3
# # df_test['geometry'] = list(zip(df_test.x, df_test.y))
# pts = list(zip(df_test.x, df_test.y))

# kd = KDTree(pts, distance_metric='Arc', radius = pysal.cg.sphere.RADIUS_EARTH_KM)
# id_nearest_point = kd.query_ball_point((center_point.x, center_point.y),radius)
# id_nearest_point
# df_cluster = df_test.iloc[id_nearest_point]
# df_cluster 

# # map
# mapobj_oneway = folium.Map([10.783284, 106.682347], zoom_start = 15, tiles='Cartodb dark_matter')
# '''
# tag = 1 (yellow):  same direction with trip_id
# tag = -1 (pink): opposite direction
# tag = 0 (red): at least 2 way
# '''
# # test 1 trip
# show_n_route(mapobj_oneway, df_lines[df_lines.trip == trip_id],1)
# show_n_route(mapobj_oneway, df_cluster,0)


In [ ]:
from pysal.cg.kdtree import KDTree
import pysal

In [ ]:
# add oneway col
df_test['oneway'] = -2
# get list trajecs
trips = list(df_lines.trip.unique())
len(trips)

In [ ]:
%%time
radius = 10*10**-3
pts = list(zip(df_test.x, df_test.y))
kd = KDTree(pts, distance_metric='Arc', radius = pysal.cg.sphere.RADIUS_EARTH_KM)
# test
while trips != []:
    trip_id = trips.pop()
    print(f'in:trip len= {len(trips)} ====== trip id = {trip_id}')
    # run
    oneway_tag, cluster_trips = detect_oneway(df_test, df_lines, kd, trip_id)
    # add tag into data
    df_lines.loc[df_lines.trip == trip_id,'oneway'] = oneway_tag
    print('======')
        

## VISUALIZING DATA

In [ ]:
import folium 

### helper functions: visualize into folium map

In [ ]:
colors = [
    'green',
    'red',
    'orange',
    'yellow',
    'pink']

def create_map_obj():
    #Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 800)
#     mapobj = folium.Map([10.778015126603638, 106.68162304214593], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj = folium.Map([10.778015126603638, 106.68162304214593], zoom_start = 15)
    mapobj.add_to(f)
    return mapobj

def add_point(mapobj, df, colors):
    #Nạp x,y từ dataframe vào list coords
    coords = list(zip(df.y, df.x))
    #Hiển thị trên mapobj
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 2, 
                            fill = True,
                            fill_opacity = 1,
                            color = colors,
                            weight = 0.05).add_to(mapobj)
           
def add_lines(mapobj, df, color):
    coords = list(zip(df.y, df.x))
    folium.PolyLine(coords, color=color, weight=2, opacity=1).add_to(mapobj)
       
    
'''
Hàm hiển thị map.
Tùy chọn:
- mapobj: bản đồ nền
- gdf: geodataframe
- start: lộ trình bắt đầu
- end: lộ trình kết thúc
- mask_type: dạng đường (1) và dạng điểm (0)
'''
def show_n_route(mapobj, gdf, mask_type, single_color=-1):
    # Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 600)
    mapobj.add_to(f)

    # Get list trajecs id
    trajecs_id = list(gdf.trip.unique())
    # Show every single trajec
    for idx, id in enumerate(trajecs_id):
        # get sub trajec
        subgdf = gdf[gdf['trip'] == id]
        color_key = idx
        # coloring with only 1 color
        if single_color != -1:
            color_key = single_color
        # coloring into mapobj
        if mask_type == 0:
            add_point(mapobj, subgdf, colors[color_key % len(colors)])
        else:
            add_lines(mapobj, subgdf, colors[color_key % len(colors)])
  
    return mapobj

def show_n_route_label(mapobj, df, mask_type):
    # Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 600)
    mapobj.add_to(f)

    # Get list label
    labels = list(df.label.unique())
    # Show every single trajec
    for idx, label in enumerate(labels):
        # get sub trajec
        subdf = df[df['label'] == label]
        # color key
        if mask_type == 0:
            if label == (len(colors)-1):
                label = 0
            add_point(mapobj, subdf, colors[ label % len(colors)])
        else:
            add_lines(mapobj, subdf, colors[ label % len(colors)])
  
    return mapobj

def show_n_route_oneway(mapobj, gdf):
    # Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 600)
    mapobj.add_to(f)

    # Get list trajecs id
    trajecs_id = list(gdf.trip.unique())
    # Show every single trajec
    for idx, id in enumerate(trajecs_id):
        # get sub trajec
        subgdf = gdf[gdf['trip'] == id]
        # color key
        oneway_tag = subgdf.iloc[0].oneway
        if oneway_tag == -1:
            oneway_tag = 1
        '''
        tag = 1:  same direction with trip_id
        tag = -1: opposite direction
        tag = 0: at least 2 way
        '''
        add_point(mapobj, subgdf, colors[int(oneway_tag) % len(colors)])

    return mapobj

### Show lines

In [ ]:
# map
mapobj1 = create_map_obj()
# # show points fisrt
# show_n_route_label(mapobj1, gdf_lines, 0)
# show lines
show_n_route(mapobj1,df_lines, 1)


### Show oneway points

In [ ]:
# map
mapobj_oneway = create_map_obj()
'''
tag = 1 (red):  same direction with trip_id
tag = -1 (red): opposite direction
tag = 0 (yellow): at least 2 way
tag = -2 (green): undefine
'''
show_n_route(mapobj_oneway,df_lines, 1, single_color=2)
show_n_route_oneway(mapobj_oneway, df_lines)


Saving output

In [ ]:
fname1 = "output/before.html"
mapobj1.save(fname1)

In [ ]:
fname_cluster = f"output/cluster_{min_cluster_size}_{epsilon*earth_radius_km}.html"
mapobj_cluster.save(fname_cluster)

In [ ]:
fname_nearest = f"output/nearest_{min_cluster_size}_{epsilon*earth_radius_km}.html"
mapobj_nearest.save(fname_nearest)

In [ ]:
fname_oneway = f"output/oneway_{day}.html"
mapobj_oneway.save(fname_oneway)

In [ ]:
df_lines.to_csv('output/oneway.csv')